# San Francisco Housing Cost Analysis

In this assignment, you will perform fundamental analysis for the San Francisco housing market to allow potential real estate investors to choose rental investment properties.

In [ ]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import numpy as np
import os
from pathlib import Path
from dotenv import load_dotenv

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("mapbox")

## Load Data

In [ ]:
# Read the census data into a Pandas DataFrame
file_path = Path("Data/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col="year")
sfo_data

- - - 

## Housing Units Per Year

In this section, you will calculate the number of housing units per year and visualize the results as a bar chart using the Pandas plot function.

**Hint:** Use the Pandas `groupby` function.

**Optional challenge:** Use the min, max, and std to scale the y limits of the chart.



In [ ]:
# Calculate the mean number of housing units per year (hint: use groupby)
housing_units = sfo_data.groupby('year').mean()
housing_units = housing_units.drop(["sale_price_sqr_foot", "gross_rent"], axis=1)
housing_units

In [ ]:
ave_min = housing_units.min()


In [ ]:
ave_max = housing_units.max()


In [ ]:
ave_std = housing_units.std()


In [ ]:
# Save the dataframe  a csv file
housing_units.to_csv()

In [ ]:
# Use the Pandas plot function to plot the average housing units per year.
# Note: You will need to manually adjust the y limit of the chart using the min and max values from above.
#housing_units.plot.bar(ylim=[370000, 400000])
housing_units.plot.bar(ylim=[ave_min[0],ave_max[0]])
# Optional Challenge: Use the min, max, and std to scale the y limits of the chart


- - - 

## Average Housing Costs in San Francisco Per Year

In this section, you will calculate the average monthly rent and the average price per square foot for each year. An investor may wish to better understand the sales price of the rental property over time. For example, a customer will want to know if they should expect an increase or decrease in the property value over time so they can determine how long to hold the rental property.  Plot the results as two line charts.

**Optional challenge:** Plot each line chart in a different color.

In [ ]:
# Calculate the average sale price per square foot and average gross rent
sqr_foot = sfo_data.groupby('year').mean()
sqr_foot = sqr_foot.drop(["housing_units", "gross_rent"], axis=1)
sqr_foot

In [ ]:
#dropping to get specified data.  gross rent data.
gross_rent = sfo_data.groupby('year').mean()
gross_rent = gross_rent.drop(["sale_price_sqr_foot", "housing_units"], axis=1)
gross_rent

In [ ]:
# Create two line charts, one to plot the average sale price per square foot and another for average montly rent
## gathering the data we parced... 
# Line chart for average sale price per square foot
sqr_foot.plot.line()
# Line chart for average montly rent
gross_rent.plot.line()

- - - 

## Average Prices by Neighborhood

In this section, you will use hvplot to create two interactive visulizations of average prices with a dropdown selector for the neighborhood. The first visualization will be a line plot showing the trend of average price per square foot over time for each neighborhood.  The second will be a line plot showing the trend of average montly rent over time for each neighborhood.

**Hint:** It will be easier to create a new DataFrame from grouping the data and calculating the mean prices for each year and neighborhood

In [ ]:
# Group by year and neighborhood and then create a new dataframe of the mean values
#gather multiple data.. 
hood_year = sfo_data.groupby(['year', 'neighborhood']).mean()
hood_year

In [ ]:
# Use hvplot to create an interactive line chart of the average price per sq ft.
# The plot should have a dropdown selector for the neighborhood
## make everything in one function. office hours why a groupby workes but do not need a function. 
hood_year['sale_price_sqr_foot'].hvplot.bar(
    title='Average Price Per Square Foot',
    xlabel= "Year",
    ylabel="housing_units",
    groupby="neighborhood")

In [ ]:
# Use hvplot to create an interactive line chart of the average monthly rent.
# The plot should have a dropdown selector for the neighborhood
hood_year['gross_rent'].hvplot.bar(
    title='Average Gross Rent',
    xlabel= "Year",
    ylabel="Average Gross Rent",
    groupby="neighborhood")

## The Top 10 Most Expensive Neighborhoods

In this section, you will need to calculate the mean sale price per square foot for each neighborhood and then sort the values to obtain the top 10 most expensive neighborhoods on average. Plot the results as a bar chart.

In [ ]:
# Getting the data from the top 10 expensive neighborhoods to own
exp_hoods = sfo_data.groupby(["neighborhood"]).mean()
ten_exp_hood = exp_hoods.sort_values('sale_price_sqr_foot', ascending=True).tail(10)
ten_exp_plot= ten_exp_hood.drop(['housing_units', 'gross_rent'], axis=1)

In [ ]:
#exp_hoods = sfo_data.reset_index().drop(['year', 'housing_units', 'gross_rent'], axis=1).set_index(['neighborhood'])
#ten_exp_hood = exp_hoods.groupby(['neighborhood']).mean().sort_values(by=['sale_price_sqr_foot'], ascending= False)
#ten_exp_hood.head(10)

#### EXTRA WAY OF DOING IT... MAYBE A WAY OF DOUBLE RESETTING INDEX. 

In [ ]:
# Plotting the data from the top 10 expensive neighborhoods
ten_exp_plot.hvplot(kind='bar', rot=90)

- - - 

## Comparing cost to purchase versus rental income

In this section, you will use `hvplot` to create an interactive visualization with a dropdown selector for the neighborhood. This visualization will feature a side-by-side comparison of average price per square foot versus average montly rent by year.  

**Hint:** Use the `hvplot` parameter, `groupby`, to create a dropdown selector for the neighborhood.

In [ ]:
# Fetch the previously generated DataFrame that was grouped by year and neighborhood
hood_P_RI = sfo_data.groupby(['year', 'neighborhood']).mean()
hood_P_RI

- - - 

In [ ]:
hood_P_RI.hvplot.bar(groupby='neighborhood', y=['sale_price_sqr_foot', 'gross_rent'], ylabel='number of housing units', xlabel='neighborhood', rot=90)

In [ ]:
print(f'Current working Directory: {Path.cwd()}')

## Neighborhood Map

In this section, you will read in neighborhoods location data and build an interactive map with the average house value per neighborhood. Use a `scatter_mapbox` from Plotly express to create the visualization. Remember, you will need your Mapbox API key for this.

### Load Location Data

In [ ]:
# Load neighborhoods coordinates data
file_path = Path('../Starter_Code/Data/neighborhoods_coordinates.csv')
hood_coor = pd.read_csv(file_path)
hood_coor

### Data Preparation

You will need to join the location data with the mean values per neighborhood.

1. Calculate the mean values for each neighborhood.

2. Join the average values with the neighborhood locations.

In [ ]:
# Calculate the mean values for each neighborhood
mean_values = sfo_data.groupby(by='neighborhood').mean()
mean_values = mean_values.sort_values('sale_price_sqr_foot', ascending=False).reset_index()
mean_values

In [ ]:
# Join the average values with the neighborhood locations
hood_locations = pd.concat([hood_coor, mean_values], axis='columns', join='inner')
hood_locations

### Mapbox Visualization

Plot the average values per neighborhood using a Plotly express `scatter_mapbox` visualization.

In [ ]:
# Set the mapbox access token
load_dotenv()
map_box_api = os.getenv("mapbox")
px.set_mapbox_access_token(map_box_api)

# Create a scatter mapbox to analyze neighborhood info
hood_map = px.scatter_mapbox(hood_locations,
    lat='Lat',
    lon='Lon',
    color='gross_rent',
    color_continuous_scale=px.colors.cyclical.IceFire,
    size_max=10,
    zoom=10,
    hover_name="Neighborhood",
    title="average sale price per square foot and gross rent in San Francisco")
hood_map.show()

- - -

## Cost Analysis - Optional Challenge

In this section, you will use Plotly express to create visualizations that investors can use to interactively filter and explore various factors related to the house value of the San Francisco's neighborhoods. 

### Create a DataFrame showing the most expensive neighborhoods in San Francisco by year

In [ ]:
# Fetch the data from all expensive neighborhoods per year.
df_expensive_neighborhoods_per_year = df_costs[df_costs["neighborhood"].isin(df_expensive_neighborhoods["neighborhood"])]
df_expensive_neighborhoods_per_year.head()

### Create a parallel coordinates plot and parallel categories plot of most expensive neighborhoods in San Francisco per year


In [ ]:
# Parallel Categories Plot
# YOUR CODE HERE!

In [ ]:
# Parallel Coordinates Plot
# YOUR CODE HERE!

### Create a sunburst chart to conduct a costs analysis of most expensive neighborhoods in San Francisco per year

In [ ]:
# Sunburst Plot
# YOUR CODE HERE!